# Publish Me: Release Assistant
The purpose of this project is to take published books sales data, then to sort it by sub-genre to allow self-publish authors to determine the highest probability for successful sales of their novel based on genre and sub-genre. The long term goal would be to design a web based dashboard that will allow the author search or filter for their sub-genre for sales numbers and to cross check with planned release dates of similar books that will be published by traditional publishing houses to prevent lower sales by releasing self-published books at or around the same time of more well known or popular authors.

### Dataset locations: 
Books_Data_Clean.csv (https://www.kaggle.com/datasets/thedevastator/books-sales-and-ratings)

book_details.csv (https://www.kaggle.com/datasets/evilspirit05/comprehensive-goodreads-book-dataset?resource=download)   

In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import nltk
# nltk.download('stopwords')
import string
import sqlite3
from matplotlib.ticker import FuncFormatter


### Data Collection and Loading: 

#### Load Data: I used Pandas to load the datasets via a CSV file. 

#### Initial Check: Please see below for the initial disply of the dataset. Then the first few rows and basic information about the dataset, noting column names, types, and missing values.

#### Selection Options:

⁜ From 'Books_Data_Clean', I dropped columns that were not needed for the analysis (index, language_code, Author_Rating, Book_average_rating, Book_ratings_count). 

⁜ The columns that I focused in 'Books_Data_Clean' were Book_Name, Author, genre, gross_sales, publisher_revenue, sale_price, sales_rank, Pulisher_ and units_sold.  

⁜ Prior to joining the data sets, I changed strings to upper and replaced special characters with ' ' to facilitate easier joining of the datasets. 

⁜ In order to differentiate genre into sub-genre, I joined 'Books_Data_Clean' with 'book_details' by book Title. 

⁜ Any books listed in the combined dataset with null values in author were dropped to make the dataset more concise with the sales data. 

In [89]:
BS = pd.read_csv("Books_Data_Clean.csv")

BS

,index,Publishing Year,Book Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross sales,publisher revenue,sale price,sales rank,Publisher,units sold
0,0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,34160.00,20496.000,4.88,1,HarperCollins Publishers,7000
1,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.50,7462.500,1.99,2,HarperCollins Publishers,6250
2,2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,genre fiction,47795.00,28677.000,8.69,3,"Amazon Digital Services, Inc.",5500
3,3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.00,24750.000,7.50,3,Hachette Book Group,5500
4,4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,genre fiction,37952.50,22771.500,7.99,4,Penguin Group (USA) LLC,4750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,1065,2014.0,Gray Mountain,John Grisham,eng,Intermediate,3.52,37379,nonfiction,104.94,62.964,0.99,1268,"Amazon Digital Services, Inc.",106
1066,1066,1989.0,The Power of One,Bryce Courtenay,eng,Excellent,4.34,57312,genre fiction,846.94,508.164,7.99,1270,Random House LLC,106
1067,1067,1930.0,The Maltese Falcon,Dashiell Hammett,eng,Intermediate,3.92,58742,genre fiction,846.94,508.164,7.99,1271,Hachette Book Group,106
1068,1068,2011.0,Night Road,Kristin Hannah,en-US,Excellent,4.17,58028,genre fiction,104.94,62.964,0.99,1272,"Amazon Digital Services, Inc.",106


In [90]:
BS.describe()

,index,Publishing Year,Book_average_rating,Book_ratings_count,gross sales,publisher revenue,sale price,sales rank,units sold
count,1070.000000,1069.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000
mean,534.500000,1971.377923,4.007000,94909.913084,1856.622944,843.281030,4.869561,611.652336,9676.980374
std,309.026698,185.080257,0.247244,31513.242518,3936.924240,2257.596743,3.559919,369.849830,15370.571306
min,0.000000,-560.000000,2.970000,27308.000000,104.940000,0.000000,0.990000,1.000000,106.000000
25%,267.250000,1985.000000,3.850000,70398.000000,372.465000,0.000000,1.990000,287.500000,551.250000
50%,534.500000,2003.000000,4.015000,89309.000000,809.745000,273.078000,3.990000,595.500000,3924.000000
75%,801.750000,2010.000000,4.170000,113906.500000,1487.957500,721.180500,6.990000,932.500000,5312.250000
max,1069.000000,2016.000000,4.770000,206792.000000,47795.000000,28677.000000,33.860000,1273.000000,61560.000000


In [91]:
BS = BS.drop(['index','language_code','Author_Rating','Book_average_rating','Book_ratings_count'], axis=1)
BS

,Publishing Year,Book Name,Author,genre,gross sales,publisher revenue,sale price,sales rank,Publisher,units sold
0,1975.0,Beowulf,"Unknown, Seamus Heaney",genre fiction,34160.00,20496.000,4.88,1,HarperCollins Publishers,7000
1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",genre fiction,12437.50,7462.500,1.99,2,HarperCollins Publishers,6250
2,2015.0,Go Set a Watchman,Harper Lee,genre fiction,47795.00,28677.000,8.69,3,"Amazon Digital Services, Inc.",5500
3,2008.0,When You Are Engulfed in Flames,David Sedaris,fiction,41250.00,24750.000,7.50,3,Hachette Book Group,5500
4,2011.0,Daughter of Smoke & Bone,Laini Taylor,genre fiction,37952.50,22771.500,7.99,4,Penguin Group (USA) LLC,4750
...,...,...,...,...,...,...,...,...,...,...
1065,2014.0,Gray Mountain,John Grisham,nonfiction,104.94,62.964,0.99,1268,"Amazon Digital Services, Inc.",106
1066,1989.0,The Power of One,Bryce Courtenay,genre fiction,846.94,508.164,7.99,1270,Random House LLC,106
1067,1930.0,The Maltese Falcon,Dashiell Hammett,genre fiction,846.94,508.164,7.99,1271,Hachette Book Group,106
1068,2011.0,Night Road,Kristin Hannah,genre fiction,104.94,62.964,0.99,1272,"Amazon Digital Services, Inc.",106


In [92]:
BS.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Publishing Year    1069 non-null   float64
 1   Book Name          1047 non-null   object 
 2   Author             1070 non-null   object 
 3   genre              1070 non-null   object 
 4   gross sales        1070 non-null   float64
 5   publisher revenue  1070 non-null   float64
 6   sale price         1070 non-null   float64
 7   sales rank         1070 non-null   int64  
 8   Publisher          1070 non-null   object 
 9   units sold         1070 non-null   int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 83.7+ KB


In [93]:
# Replace spaces with underscores in column names
BS.columns = BS.columns.str.replace(' ', '_')
BS.head()

,Publishing_Year,Book_Name,Author,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher_,units_sold
0,1975.0,Beowulf,"Unknown, Seamus Heaney",genre fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",genre fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2015.0,Go Set a Watchman,Harper Lee,genre fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,2008.0,When You Are Engulfed in Flames,David Sedaris,fiction,41250.0,24750.0,7.50,3,Hachette Book Group,5500
4,2011.0,Daughter of Smoke & Bone,Laini Taylor,genre fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


In [94]:
# change string to upper single column
#BS['Book Name'] = BS['Book Name'].str.upper() 

# change string to upper for multiple columns
BS = BS.assign(Book_Name=BS['Book_Name'].str.upper(), 
               Author=BS['Author'].str.upper(),
               genre=BS['genre'].str.upper(),
               Publisher_=BS['Publisher_'].str.upper())
BS.head()

,Publishing_Year,Book_Name,Author,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher_,units_sold
0,1975.0,BEOWULF,"UNKNOWN, SEAMUS HEANEY",GENRE FICTION,34160.0,20496.0,4.88,1,HARPERCOLLINS PUBLISHERS,7000
1,1987.0,BATMAN: YEAR ONE,"FRANK MILLER, DAVID MAZZUCCHELLI, RICHMOND LEW...",GENRE FICTION,12437.5,7462.5,1.99,2,HARPERCOLLINS PUBLISHERS,6250
2,2015.0,GO SET A WATCHMAN,HARPER LEE,GENRE FICTION,47795.0,28677.0,8.69,3,"AMAZON DIGITAL SERVICES, INC.",5500
3,2008.0,WHEN YOU ARE ENGULFED IN FLAMES,DAVID SEDARIS,FICTION,41250.0,24750.0,7.50,3,HACHETTE BOOK GROUP,5500
4,2011.0,DAUGHTER OF SMOKE & BONE,LAINI TAYLOR,GENRE FICTION,37952.5,22771.5,7.99,4,PENGUIN GROUP (USA) LLC,4750


In [95]:
#remove special characters from single column
BS['Book_Name'] = BS['Book_Name'].str.replace('\W', ' ', regex=True)
BS['Author'] = BS['Author'].str.replace('\W', ' ', regex=True)
BS['genre'] = BS['genre'].str.replace('\W', ' ', regex=True)
BS['Publisher_'] = BS['Publisher_'].str.replace('\W', ' ', regex=True)
# replace special character with ' ' for all columns
#df = df.replace(r'[^0-9a-zA-Z ]', '', regex=True).replace("'", '')
BS.head(10)

,Publishing_Year,Book_Name,Author,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher_,units_sold
0,1975.0,BEOWULF,UNKNOWN SEAMUS HEANEY,GENRE FICTION,34160.00,20496.000,4.88,1,HARPERCOLLINS PUBLISHERS,7000
1,1987.0,BATMAN YEAR ONE,FRANK MILLER DAVID MAZZUCCHELLI RICHMOND LEW...,GENRE FICTION,12437.50,7462.500,1.99,2,HARPERCOLLINS PUBLISHERS,6250
2,2015.0,GO SET A WATCHMAN,HARPER LEE,GENRE FICTION,47795.00,28677.000,8.69,3,AMAZON DIGITAL SERVICES INC,5500
3,2008.0,WHEN YOU ARE ENGULFED IN FLAMES,DAVID SEDARIS,FICTION,41250.00,24750.000,7.50,3,HACHETTE BOOK GROUP,5500
4,2011.0,DAUGHTER OF SMOKE BONE,LAINI TAYLOR,GENRE FICTION,37952.50,22771.500,7.99,4,PENGUIN GROUP USA LLC,4750
5,2015.0,RED QUEEN,VICTORIA AVEYARD,GENRE FICTION,19960.00,0.000,4.99,5,AMAZON DIGITAL SERVICES INC,4000
6,2011.0,THE POWER OF HABIT,CHARLES DUHIGG,GENRE FICTION,27491.67,16495.002,6.99,6,HARPERCOLLINS PUBLISHERS,3933
7,1994.0,MIDNIGHT IN THE GARDEN OF GOOD AND EVIL,JOHN BERENDT,NONFICTION,26182.00,15709.200,6.89,8,HACHETTE BOOK GROUP,3800
8,2012.0,HOPELESS,COLLEEN HOOVER,GENRE FICTION,26093.67,15656.202,6.99,9,HARPERCOLLINS PUBLISHERS,3733
9,1905.0,A LITTLE PRINCESS,FRANCES HODGSON BURNETT NANCY BOND,GENRE FICTION,23792.34,14275.404,6.49,10,RANDOM HOUSE LLC,3666


In [96]:
#replace genre fiction with fiction
BS['genre'] = BS['genre'].replace('GENRE FICTION','FICTION')
BS.head(10)

,Publishing_Year,Book_Name,Author,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher_,units_sold
0,1975.0,BEOWULF,UNKNOWN SEAMUS HEANEY,FICTION,34160.00,20496.000,4.88,1,HARPERCOLLINS PUBLISHERS,7000
1,1987.0,BATMAN YEAR ONE,FRANK MILLER DAVID MAZZUCCHELLI RICHMOND LEW...,FICTION,12437.50,7462.500,1.99,2,HARPERCOLLINS PUBLISHERS,6250
2,2015.0,GO SET A WATCHMAN,HARPER LEE,FICTION,47795.00,28677.000,8.69,3,AMAZON DIGITAL SERVICES INC,5500
3,2008.0,WHEN YOU ARE ENGULFED IN FLAMES,DAVID SEDARIS,FICTION,41250.00,24750.000,7.50,3,HACHETTE BOOK GROUP,5500
4,2011.0,DAUGHTER OF SMOKE BONE,LAINI TAYLOR,FICTION,37952.50,22771.500,7.99,4,PENGUIN GROUP USA LLC,4750
5,2015.0,RED QUEEN,VICTORIA AVEYARD,FICTION,19960.00,0.000,4.99,5,AMAZON DIGITAL SERVICES INC,4000
6,2011.0,THE POWER OF HABIT,CHARLES DUHIGG,FICTION,27491.67,16495.002,6.99,6,HARPERCOLLINS PUBLISHERS,3933
7,1994.0,MIDNIGHT IN THE GARDEN OF GOOD AND EVIL,JOHN BERENDT,NONFICTION,26182.00,15709.200,6.89,8,HACHETTE BOOK GROUP,3800
8,2012.0,HOPELESS,COLLEEN HOOVER,FICTION,26093.67,15656.202,6.99,9,HARPERCOLLINS PUBLISHERS,3733
9,1905.0,A LITTLE PRINCESS,FRANCES HODGSON BURNETT NANCY BOND,FICTION,23792.34,14275.404,6.49,10,RANDOM HOUSE LLC,3666


In [97]:
Book_Details = pd.read_csv ("book_details.csv")
Book_Details

,title,url,description,genres
0,To Kill a Mockingbird,https://www.goodreads.com/book/show/2657.To_Ki...,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ..."
1,Harry Potter and the Philosopher’s Stone,https://www.goodreads.com/book/show/72193.Harr...,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',..."
2,Pride and Prejudice,https://www.goodreads.com/book/show/1885.Pride...,Alternate cover edition of ISBN 9780679783268S...,"['Classics', 'Fiction', 'Romance', 'Historical..."
3,The Diary of a Young Girl,https://www.goodreads.com/book/show/48855.The_...,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp..."
4,Animal Farm,https://www.goodreads.com/book/show/170448.Ani...,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',..."
...,...,...,...,...
6308,The Blank Slate: The Modern Denial of Human Na...,https://www.goodreads.com/book/show/5752.The_B...,"In The Blank Slate, Steven Pinker explores the...","['Psychology', 'Science', 'Nonfiction', 'Philo..."
6309,La leyenda del Rey Errante,https://www.goodreads.com/book/show/1146745.La...,"Cuenta la historia de Walid, un príncipe árabe...","['Fantasy', 'Historical Fiction', 'Fiction', '..."
6310,"This Way for the Gas, Ladies and Gentlemen",https://www.goodreads.com/book/show/228244.Thi...,Tadeusz Borowski’s concentration camp stories ...,"['History', 'Short Stories', 'Holocaust', 'Non..."
6311,The Warded Man,https://www.goodreads.com/book/show/3428935-th...,"As darkness falls after sunset, the corelings ...","['Fantasy', 'Fiction', 'Epic Fantasy', 'High F..."


In [98]:
Book_Details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6313 entries, 0 to 6312
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        6313 non-null   object
 1   url          6313 non-null   object
 2   description  6274 non-null   object
 3   genres       6313 non-null   object
dtypes: object(4)
memory usage: 197.4+ KB


In [99]:
Book_Details.describe()

,title,url,description,genres
count,6313,6313,6274,6313
unique,6183,6313,6252,5889
top,The Hunger Games,https://www.goodreads.com/book/show/18361450-r...,Lost in Yaba is a true story about an expat wh...,[]
freq,3,1,2,163


In [100]:
# change string to upper for multiple columns
Book_Details = Book_Details.assign(title=Book_Details['title'].str.upper()) 
Book_Details.head()

,title,url,description,genres
0,TO KILL A MOCKINGBIRD,https://www.goodreads.com/book/show/2657.To_Ki...,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ..."
1,HARRY POTTER AND THE PHILOSOPHER’S STONE,https://www.goodreads.com/book/show/72193.Harr...,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',..."
2,PRIDE AND PREJUDICE,https://www.goodreads.com/book/show/1885.Pride...,Alternate cover edition of ISBN 9780679783268S...,"['Classics', 'Fiction', 'Romance', 'Historical..."
3,THE DIARY OF A YOUNG GIRL,https://www.goodreads.com/book/show/48855.The_...,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp..."
4,ANIMAL FARM,https://www.goodreads.com/book/show/170448.Ani...,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',..."


In [101]:
#remove special characters from columns
Book_Details['title'] = Book_Details['title'].str.replace('\W', ' ', regex=True)
Book_Details.head(10)

,title,url,description,genres
0,TO KILL A MOCKINGBIRD,https://www.goodreads.com/book/show/2657.To_Ki...,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ..."
1,HARRY POTTER AND THE PHILOSOPHER S STONE,https://www.goodreads.com/book/show/72193.Harr...,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',..."
2,PRIDE AND PREJUDICE,https://www.goodreads.com/book/show/1885.Pride...,Alternate cover edition of ISBN 9780679783268S...,"['Classics', 'Fiction', 'Romance', 'Historical..."
3,THE DIARY OF A YOUNG GIRL,https://www.goodreads.com/book/show/48855.The_...,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp..."
4,ANIMAL FARM,https://www.goodreads.com/book/show/170448.Ani...,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',..."
5,THE LITTLE PRINCE,https://www.goodreads.com/book/show/157993.The...,A pilot stranded in the desert awakes one morn...,"['Classics', 'Fiction', 'Fantasy', 'Childrens'..."
6,1984,https://www.goodreads.com/book/show/61439040-1984,The new novel by George Orwell is the major wo...,"['Classics', 'Fiction', 'Science Fiction', 'Dy..."
7,THE GREAT GATSBY,https://www.goodreads.com/book/show/4671.The_G...,Alternate Cover Edition ISBN: 0743273567 (ISBN...,"['Classics', 'Fiction', 'School', 'Historical ..."
8,THE CATCHER IN THE RYE,https://www.goodreads.com/book/show/5107.The_C...,It's Christmas time and Holden Caulfield has j...,"['Classics', 'Fiction', 'Young Adult', 'Litera..."
9,THE LORD OF THE RINGS,https://www.goodreads.com/book/show/33.The_Lor...,"One Ring to rule them all, One Ring to find th...","['Fantasy', 'Classics', 'Fiction', 'Adventure'..."


In [102]:
#split genres column into multiple sub-genre columns 
Book_Details[['Sub_Genre_1', 'Sub_Genre_2', 'Sub_Genre_3','Sub_Genre_4','Sub_Genre_5','Sub_Genre_6','Sub_Genre_7','Sub_Genre_8','Sub_Genre_9','Sub_Genre_10']] = Book_Details['genres'].str.split(',', expand=True)
Book_Details


,title,url,description,genres,Sub_Genre_1,Sub_Genre_2,Sub_Genre_3,Sub_Genre_4,Sub_Genre_5,Sub_Genre_6,Sub_Genre_7,Sub_Genre_8,Sub_Genre_9,Sub_Genre_10
0,TO KILL A MOCKINGBIRD,https://www.goodreads.com/book/show/2657.To_Ki...,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",['Classics','Fiction','Historical Fiction','School','Literature','Young Adult','Historical','Read For School','Novels','High School']
1,HARRY POTTER AND THE PHILOSOPHER S STONE,https://www.goodreads.com/book/show/72193.Harr...,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",['Fantasy','Fiction','Young Adult','Magic','Childrens','Middle Grade','Classics','Adventure','Audiobook','Science Fiction Fantasy']
2,PRIDE AND PREJUDICE,https://www.goodreads.com/book/show/1885.Pride...,Alternate cover edition of ISBN 9780679783268S...,"['Classics', 'Fiction', 'Romance', 'Historical...",['Classics','Fiction','Romance','Historical Fiction','Literature','Historical','Audiobook','Novels','Historical Romance','Adult']
3,THE DIARY OF A YOUNG GIRL,https://www.goodreads.com/book/show/48855.The_...,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",['Classics','Nonfiction','History','Biography','Memoir','Historical','Holocaust'],None,None,None
4,ANIMAL FARM,https://www.goodreads.com/book/show/170448.Ani...,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",['Classics','Fiction','Dystopia','Fantasy','Politics','Literature','School','Science Fiction','Novels','Read For School']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6308,THE BLANK SLATE THE MODERN DENIAL OF HUMAN NA...,https://www.goodreads.com/book/show/5752.The_B...,"In The Blank Slate, Steven Pinker explores the...","['Psychology', 'Science', 'Nonfiction', 'Philo...",['Psychology','Science','Nonfiction','Philosophy','Sociology','Biology','Politics','Evolution','Anthropology','Neuroscience']
6309,LA LEYENDA DEL REY ERRANTE,https://www.goodreads.com/book/show/1146745.La...,"Cuenta la historia de Walid, un príncipe árabe...","['Fantasy', 'Historical Fiction', 'Fiction', '...",['Fantasy','Historical Fiction','Fiction','Young Adult','Adventure','Historical','Middle Grade','Spanish Literature','Childrens','Teen']
6310,THIS WAY FOR THE GAS LADIES AND GENTLEMEN,https://www.goodreads.com/book/show/228244.Thi...,Tadeusz Borowski’s concentration camp stories ...,"['History', 'Short Stories', 'Holocaust', 'Non...",['History','Short Stories','Holocaust','Nonfiction','War','Polish Literature','Classics'],None,None,None
6311,THE WARDED MAN,https://www.goodreads.com/book/show/3428935-th...,"As darkness falls after sunset, the corelings ...","['Fantasy', 'Fiction', 'Epic Fantasy', 'High F...",['Fantasy','Fiction','Epic Fantasy','High Fantasy','Magic','Demons','Adventure','Dark Fantasy','Science Fiction Fantasy','Audiobook']


In [103]:
#drop genres then remove all special characters from Sub_Genre columns and change strings to CAPS
Book_Details = Book_Details.drop('genres', axis=1)

Book_Details = Book_Details.assign(Sub_Genre_1=Book_Details['Sub_Genre_1'].str.upper(), 
               Sub_Genre_2=Book_Details['Sub_Genre_2'].str.upper(),
               Sub_Genre_3=Book_Details['Sub_Genre_3'].str.upper(),
               Sub_Genre_4=Book_Details['Sub_Genre_4'].str.upper(),
               Sub_Genre_5=Book_Details['Sub_Genre_5'].str.upper(),
               Sub_Genre_6=Book_Details['Sub_Genre_6'].str.upper(),
               Sub_Genre_7=Book_Details['Sub_Genre_7'].str.upper(),
               Sub_Genre_8=Book_Details['Sub_Genre_8'].str.upper(),
               Sub_Genre_9=Book_Details['Sub_Genre_9'].str.upper(),
               Sub_Genre_10=Book_Details['Sub_Genre_10'].str.upper())

Book_Details['Sub_Genre_1'] = Book_Details['Sub_Genre_1'].str.replace('\W', ' ', regex=True)
Book_Details['Sub_Genre_2'] = Book_Details['Sub_Genre_2'].str.replace('\W', ' ', regex=True)
Book_Details['Sub_Genre_3'] = Book_Details['Sub_Genre_3'].str.replace('\W', ' ', regex=True)
Book_Details['Sub_Genre_4'] = Book_Details['Sub_Genre_4'].str.replace('\W', ' ', regex=True)
Book_Details['Sub_Genre_5'] = Book_Details['Sub_Genre_5'].str.replace('\W', ' ', regex=True)
Book_Details['Sub_Genre_6'] = Book_Details['Sub_Genre_6'].str.replace('\W', ' ', regex=True)
Book_Details['Sub_Genre_7'] = Book_Details['Sub_Genre_7'].str.replace('\W', ' ', regex=True)
Book_Details['Sub_Genre_8'] = Book_Details['Sub_Genre_8'].str.replace('\W', ' ', regex=True)
Book_Details['Sub_Genre_9'] = Book_Details['Sub_Genre_9'].str.replace('\W', ' ', regex=True)
Book_Details['Sub_Genre_10'] = Book_Details['Sub_Genre_10'].str.replace('\W', ' ', regex=True)
Book_Details

,title,url,description,Sub_Genre_1,Sub_Genre_2,Sub_Genre_3,Sub_Genre_4,Sub_Genre_5,Sub_Genre_6,Sub_Genre_7,Sub_Genre_8,Sub_Genre_9,Sub_Genre_10
0,TO KILL A MOCKINGBIRD,https://www.goodreads.com/book/show/2657.To_Ki...,The unforgettable novel of a childhood in a sl...,CLASSICS,FICTION,HISTORICAL FICTION,SCHOOL,LITERATURE,YOUNG ADULT,HISTORICAL,READ FOR SCHOOL,NOVELS,HIGH SCHOOL
1,HARRY POTTER AND THE PHILOSOPHER S STONE,https://www.goodreads.com/book/show/72193.Harr...,Harry Potter thinks he is an ordinary boy - un...,FANTASY,FICTION,YOUNG ADULT,MAGIC,CHILDRENS,MIDDLE GRADE,CLASSICS,ADVENTURE,AUDIOBOOK,SCIENCE FICTION FANTASY
2,PRIDE AND PREJUDICE,https://www.goodreads.com/book/show/1885.Pride...,Alternate cover edition of ISBN 9780679783268S...,CLASSICS,FICTION,ROMANCE,HISTORICAL FICTION,LITERATURE,HISTORICAL,AUDIOBOOK,NOVELS,HISTORICAL ROMANCE,ADULT
3,THE DIARY OF A YOUNG GIRL,https://www.goodreads.com/book/show/48855.The_...,Discovered in the attic in which she spent the...,CLASSICS,NONFICTION,HISTORY,BIOGRAPHY,MEMOIR,HISTORICAL,HOLOCAUST,None,None,None
4,ANIMAL FARM,https://www.goodreads.com/book/show/170448.Ani...,Librarian's note: There is an Alternate Cover ...,CLASSICS,FICTION,DYSTOPIA,FANTASY,POLITICS,LITERATURE,SCHOOL,SCIENCE FICTION,NOVELS,READ FOR SCHOOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6308,THE BLANK SLATE THE MODERN DENIAL OF HUMAN NA...,https://www.goodreads.com/book/show/5752.The_B...,"In The Blank Slate, Steven Pinker explores the...",PSYCHOLOGY,SCIENCE,NONFICTION,PHILOSOPHY,SOCIOLOGY,BIOLOGY,POLITICS,EVOLUTION,ANTHROPOLOGY,NEUROSCIENCE
6309,LA LEYENDA DEL REY ERRANTE,https://www.goodreads.com/book/show/1146745.La...,"Cuenta la historia de Walid, un príncipe árabe...",FANTASY,HISTORICAL FICTION,FICTION,YOUNG ADULT,ADVENTURE,HISTORICAL,MIDDLE GRADE,SPANISH LITERATURE,CHILDRENS,TEEN
6310,THIS WAY FOR THE GAS LADIES AND GENTLEMEN,https://www.goodreads.com/book/show/228244.Thi...,Tadeusz Borowski’s concentration camp stories ...,HISTORY,SHORT STORIES,HOLOCAUST,NONFICTION,WAR,POLISH LITERATURE,CLASSICS,None,None,None
6311,THE WARDED MAN,https://www.goodreads.com/book/show/3428935-th...,"As darkness falls after sunset, the corelings ...",FANTASY,FICTION,EPIC FANTASY,HIGH FANTASY,MAGIC,DEMONS,ADVENTURE,DARK FANTASY,SCIENCE FICTION FANTASY,AUDIOBOOK


In [105]:
# Renaming columns
Book_Details.rename(columns={'title': 'Book_Name'}, inplace=True)
Book_Details.head()

,Book_Name,url,description,Sub_Genre_1,Sub_Genre_2,Sub_Genre_3,Sub_Genre_4,Sub_Genre_5,Sub_Genre_6,Sub_Genre_7,Sub_Genre_8,Sub_Genre_9,Sub_Genre_10
0,TO KILL A MOCKINGBIRD,https://www.goodreads.com/book/show/2657.To_Ki...,The unforgettable novel of a childhood in a sl...,CLASSICS,FICTION,HISTORICAL FICTION,SCHOOL,LITERATURE,YOUNG ADULT,HISTORICAL,READ FOR SCHOOL,NOVELS,HIGH SCHOOL
1,HARRY POTTER AND THE PHILOSOPHER S STONE,https://www.goodreads.com/book/show/72193.Harr...,Harry Potter thinks he is an ordinary boy - un...,FANTASY,FICTION,YOUNG ADULT,MAGIC,CHILDRENS,MIDDLE GRADE,CLASSICS,ADVENTURE,AUDIOBOOK,SCIENCE FICTION FANTASY
2,PRIDE AND PREJUDICE,https://www.goodreads.com/book/show/1885.Pride...,Alternate cover edition of ISBN 9780679783268S...,CLASSICS,FICTION,ROMANCE,HISTORICAL FICTION,LITERATURE,HISTORICAL,AUDIOBOOK,NOVELS,HISTORICAL ROMANCE,ADULT
3,THE DIARY OF A YOUNG GIRL,https://www.goodreads.com/book/show/48855.The_...,Discovered in the attic in which she spent the...,CLASSICS,NONFICTION,HISTORY,BIOGRAPHY,MEMOIR,HISTORICAL,HOLOCAUST,None,None,None
4,ANIMAL FARM,https://www.goodreads.com/book/show/170448.Ani...,Librarian's note: There is an Alternate Cover ...,CLASSICS,FICTION,DYSTOPIA,FANTASY,POLITICS,LITERATURE,SCHOOL,SCIENCE FICTION,NOVELS,READ FOR SCHOOL


In [108]:
#Join datasets via title
# using .merge() function
Full_Book_Details = pd.merge(Book_Details, BS, on='Book_Name')
Full_Book_Details


,Book_Name,url,description,Sub_Genre_1,Sub_Genre_2,Sub_Genre_3,Sub_Genre_4,Sub_Genre_5,Sub_Genre_6,Sub_Genre_7,...,Sub_Genre_10,Publishing_Year,Author,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher_,units_sold
0,THE LITTLE HOUSE COLLECTION,https://www.goodreads.com/book/show/114345.The...,This nine-book paperback box set of the classi...,CLASSICS,HISTORICAL FICTION,CHILDRENS,FICTION,YOUNG ADULT,HISTORICAL,MIDDLE GRADE,...,KIDS,1953.0,LAURA INGALLS WILDER GARTH WILLIAMS,FICTION,5195.91,3117.546,11.47,228,RANDOM HOUSE LLC,453
1,THE VELVETEEN RABBIT,https://www.goodreads.com/book/show/144974.The...,"Nursery magic is very strange and wonderful, a...",CLASSICS,CHILDRENS,FICTION,PICTURE BOOKS,FANTASY,ANIMALS,YOUNG ADULT,...,MIDDLE GRADE,1922.0,MARGERY WILLIAMS BIANCO WILLIAM NICHOLSON,FICTION,21797.82,13078.692,6.54,15,RANDOM HOUSE LLC,3333
2,LOVE YOU FOREVER,https://www.goodreads.com/book/show/310259.Lov...,An extraordinarily different story by Robert M...,PICTURE BOOKS,CHILDRENS,FICTION,CLASSICS,FAMILY,LOVE,KIDS,...,JUVENILE,1986.0,ROBERT MUNSCH SHEILA MCGRAW,FICTION,1309.77,785.862,0.99,79,AMAZON DIGITAL SERVICES INC,1323
3,JONATHAN LIVINGSTON SEAGULL,https://www.goodreads.com/book/show/71728.Jona...,This is a story for people who follow their he...,FICTION,CLASSICS,PHILOSOPHY,FANTASY,SPIRITUALITY,INSPIRATIONAL,SELF HELP,...,ANIMALS,1970.0,RICHARD BACH RUSSELL MUNSON,FICTION,6751.47,0.000,4.99,77,AMAZON DIGITAL SERVICES INC,1353
4,THE COMPLETE SHERLOCK HOLMES,https://www.goodreads.com/book/show/188572.The...,A study in scarlet --The sign of four --Advent...,CLASSICS,MYSTERY,FICTION,CRIME,SHORT STORIES,DETECTIVE,LITERATURE,...,HISTORICAL FICTION,1894.0,ARTHUR CONAN DOYLE,FICTION,3180.02,1908.012,7.99,261,PENGUIN GROUP USA LLC,398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,THE AMITYVILLE HORROR,https://www.goodreads.com/book/show/293101.The...,28 Days of Terror in a House Possessed by Evil...,HORROR,FICTION,PARANORMAL,THRILLER,CLASSICS,AUDIOBOOK,SUPERNATURAL,...,GHOSTS,1977.0,JAY ANSON,FICTION,1191.26,714.756,8.89,684,HACHETTE BOOK GROUP,52056
691,THE BONE CLOCKS,https://www.goodreads.com/book/show/20819685-t...,"Following a scalding row with her mother, fift...",FICTION,FANTASY,SCIENCE FICTION,LITERARY FICTION,MAGICAL REALISM,CONTEMPORARY,MYSTERY,...,ADULT,2014.0,DAVID MITCHELL,FICTION,781.96,469.176,6.92,1001,RANDOM HOUSE LLC,2862
692,RULES OF CIVILITY,https://www.goodreads.com/book/show/10054335-r...,This sophisticated and entertaining first nove...,FICTION,HISTORICAL FICTION,HISTORICAL,NEW YORK,LITERARY FICTION,AUDIOBOOK,BOOK CLUB,...,ADULT,2011.0,AMOR TOWLES,FICTION,712.81,427.686,5.99,760,PENGUIN GROUP USA LLC,40824
693,THE BLACK DAHLIA,https://www.goodreads.com/book/show/21704.The_...,"On January 15, 1947, the torture-ravished body...",MYSTERY,FICTION,CRIME,NOIR,HISTORICAL FICTION,TRUE CRIME,THRILLER,...,HISTORICAL,1987.0,JAMES ELLROY,FICTION,1088.91,653.346,9.99,1127,RANDOM HOUSE LLC,4280


In [110]:
# getting union  of keys
res2 = pd.merge(Book_Details, BS, how='outer', on=['Book_Name', 'Book_Name'])
 
res2


,Book_Name,url,description,Sub_Genre_1,Sub_Genre_2,Sub_Genre_3,Sub_Genre_4,Sub_Genre_5,Sub_Genre_6,Sub_Genre_7,...,Sub_Genre_10,Publishing_Year,Author,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher_,units_sold
0,A MONSTER CALLS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2011.0,PATRICK NESS JIM KAY SIOBHAN DOWD,FICTION,1842.45,1105.470,10.65,505.0,RANDOM HOUSE LLC,4671.0
1,A PROBLEM FROM HELL AMERICA AND THE AGE OF ...,https://www.goodreads.com/book/show/368731._A_...,Former UN Ambassador Samantha Power's Pulitzer...,NONFICTION,HISTORY,POLITICS,WAR,AFRICA,AMERICAN HISTORY,INTERNATIONAL RELATIONS,...,WORLD HISTORY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ANIMAL FARM 1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1950.0,GEORGE ORWELL CHRISTOPHER HITCHENS,FICTION,267.30,0.000,0.99,338.0,AMAZON DIGITAL SERVICES INC,270.0
3,GIRLBOSS,https://www.goodreads.com/book/show/18667945-g...,‘In the New York Times bestseller that the Was...,NONFICTION,BUSINESS,MEMOIR,FEMINISM,SELF HELP,BIOGRAPHY,AUDIOBOOK,...,AUTOBIOGRAPHY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NIGHT MOTHER,https://www.goodreads.com/book/show/50544._nig...,"'night, Mother is a taut and fluid drama that ...",PLAYS,DRAMA,THEATRE,FICTION,CLASSICS,20TH CENTURY,MENTAL HEALTH,...,COLLEGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2009.0,ROBERTO AGUIRRE SACASA STEPHEN KING MIKE PER...,FICTION,838.88,503.328,7.49,1043.0,PENGUIN GROUP USA LLC,4440.0
6707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2008.0,BRENT WEEKS,FICTION,442.89,0.000,3.99,1054.0,AMAZON DIGITAL SERVICES INC,4400.0
6708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1937.0,LAURA INGALLS WILDER GARTH WILLIAMS,FICTION,516.66,309.996,4.74,1131.0,HARPERCOLLINS PUBLISHERS,4280.0
6709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2016.0,JODI PICOULT,FICTION,1306.91,784.146,11.99,1144.0,PENGUIN GROUP USA LLC,4240.0
